# Testing Sequence Dataloading

**Authorship:**
Adam Klie, *03/02/2022*
***
**Description:**
Notebook for testing out PyTorch DataLoading for SeqDataSet and SeqDataModule classes

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li>Add test cases for each step</li></b>
    <b><li>Feel like initial loading will break given certain data</li></b>
    </ul>
</div>

In [52]:
# Classics
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

import eugene

In [81]:
# Define directories
OLS_TSV = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/2021_OLS_Library.tsv"
NUMPY_OHE = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/ohe_seq/0.09-0.4_X-train-0.9_ohe-seq.npy"
FASTA_SEQS = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/fasta/0.09-0.4_X-test-0.1_fasta.fa"
BINARY_TARGET = "/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/binary/0.09-0.4_y-train-0.9_binary.txt"

# SeqDataset, DataLoader, SeqDataModule Classes
PyTorch Dataset class for loading sequence data. Here tare the steps for loading:
 1. Load the dataset from files of different supported types using functions from `datasets._io.py`
 2. Generate an `SeqDataset` object from sequences and targets
     - Pass in the seqs and targets
     - Compose `torchvision.transforms`
 3. Pass the dataset to DataLoader
 4. PyTorch Lightning DataModule class for MPRA data that allows for asbtracting most of the dataloading process. These DataModules can be passed straight to trainers for model training.

# Load a random dataset from a CSV file using `load_csv`

In [86]:
from eugene.datasets import load_csv
from eugene.dataloading.dataloaders import SeqDataset
from eugene.dataloading import Augment, ReverseComplement, OneHotEncode, ToTensor
from torchvision import transforms

# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load_csv("test_100seqs_66/test_seqs.tsv", seq_col="SEQ", name_col="NAME", target_col="LABEL", rev_comp=True)
print(names[0], len(seqs), seqs[0][:5], len(rev_seqs), rev_seqs[0][-5:], targets[0])

# Compose different data transforms for this particular load
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), ReverseComplement(ohe_encoded=False), OneHotEncode(), ToTensor(transpose=False)
])

# Instantiate a Dataset
test_dataset = SeqDataset(seqs, names=names, targets=targets, transform=data_transform)
print(test_dataset[0][0], test_dataset[0][1][:5], test_dataset[0][2][:5], test_dataset[0][3])

# Check the Dataset Class
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    print(i, sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

# Instantiate a DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

# Check the DataLoader
for i_batch, sample_batched in enumerate(test_dataloader):
    print(i_batch, sample_batched[1].size(), sample_batched[2].size(), sample_batched[3].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

# SeqDataModule
test_datamod = SeqDataModule("test_100seqs_66/test_seqs.tsv")

seq001 100 GTAGG 100 CCTAC 0.0
tensor([115., 101., 113.,  48.,  48.,  49.]) tensor([[0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]]) tensor([[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]]) tensor(0.)
0 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.)
1 torch.Size([66, 4]) torch.Size([66, 4]) tensor(1.)
2 torch.Size([66, 4]) torch.Size([66, 4]) tensor(1.)
3 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.)
0 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
1 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
2 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
3 torch.Size([4, 66, 4]) torch.Size([4, 66, 4]) torch.Size([4])


# Load real `ols` dataset from `datasets` module

In [98]:
# Load from `datasets` module
names, seqs, rev_seqs, targets = eugene.datasets.ols()
print(names[0], len(seqs), seqs[0][:5], rev_seqs, targets[0])

# Compose different data transforms for this particular load
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), ReverseComplement(ohe_encoded=False), OneHotEncode(), ToTensor(transpose=False)
])

# Instantiate a Dataset
test_dataset = SeqDataset(seqs, names=names, targets=targets, transform=data_transform)
print(test_dataset[0][0], test_dataset[0][1][:5], test_dataset[0][2][:5], test_dataset[0][3])

# Check the Dataset Class
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    print(i, sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

# Instantiate a DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=0)

# Check the DataLoader
for i_batch, sample_batched in enumerate(test_dataloader):
    print(i_batch, sample_batched[1].size(), sample_batched[2].size(), sample_batched[3].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

# SeqDataModule
# Can we go from SeqDataset ro SeqDataModule?

S1-G1R-S2-E1F-S3-E2F-S4-G2R-S5-G3F-S6 460800 CATCT None 0.6117667250932141
tensor([83., 49., 45., 71., 49., 82., 45., 83., 50., 45., 69., 49., 70., 45.,
        83., 51., 45., 69., 50., 70., 45., 83., 52., 45., 71., 50., 82., 45.,
        83., 53., 45., 71., 51., 70., 45., 83., 54.]) tensor([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]]) tensor([[0., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]]) tensor(0.6118)
0 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.6118)
1 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.3130)
2 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.3871)
3 torch.Size([66, 4]) torch.Size([66, 4]) tensor(0.)
0 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
1 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
2 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
3 torch.Size(

# Load a random dataset from Numpy arrays using `load_numpy`

In [99]:
from eugene.datasets import load_numpy

# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load_numpy("../data/2021_OLS_Library/ohe_seq/0.09-0.4_X-all_ohe-seq.npy",
                                            target_file="../data/2021_OLS_Library/binary/0.09-0.4_y-all_binary.txt",
                                            is_target_text=True)
names, seqs[0][:5], rev_seqs, targets[0]

# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load_numpy("test_100seqs_1000/test_ohe_seqs.npy", 
                                            target_file="test_100seqs_66/test_labels.npy",
                                            names_file="test_100seqs_66/test_ids.npy", 
                                            rev_seq_file="test_100seqs_66/test_rev_ohe_seqs.npy")
len(names), names[0], len(seqs), seqs[0][:5], len(rev_seqs), rev_seqs[0][-5:], targets[0]

# Compose different data transforms for this particular load
data_transform = transforms.Compose([ToTensor(transpose=False)])

# Instantiate a Dataset
test_dataset = SeqDataset(seqs, names=names, rev_seqs=rev_seqs, transform=data_transform)

# Check the Dataset Class
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    print(i, sample[0].size(), sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

0 torch.Size([6]) torch.Size([1000, 4]) torch.Size([66, 4]) tensor([-1.])
1 torch.Size([6]) torch.Size([1000, 4]) torch.Size([66, 4]) tensor([-1.])
2 torch.Size([6]) torch.Size([1000, 4]) torch.Size([66, 4]) tensor([-1.])
3 torch.Size([6]) torch.Size([1000, 4]) torch.Size([66, 4]) tensor([-1.])


# Load from Fasta

In [41]:
from eugene.datasets import load_fasta 

# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load_fasta("test_100seqs_66/test_seqs.fa", "test_100seqs_66/test_labels.npy", rev_comp=True)
names[0], seqs[0], rev_seqs[0], targets[0]

# Compose different data transforms for this particular load
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), OneHotEncode(), ToTensor(transpose=False)
])

# Instantiate a Dataset
test_dataset = SeqDataset(seqs, names=names, targets=targets, rev_seqs=rev_seqs, transform=data_transform)

# Check the Dataset Class
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    print(i, sample[0].size(), sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

## Load dataset

In [72]:
# Import the laod function
from eugene.datasets import load
from torch.utils.data import DataLoader
from eugene.dataloading.dataloaders import SeqDataModule

# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load("test_100seqs_66/test_ohe_seqs.npy", names_file="test_100seqs_66/test_ids.npy", target_file="test_100seqs_66/test_labels.npy", rev_seq_file="test_100seqs_66/test_rev_ohe_seqs.npy")
len(names), names[0], len(seqs), seqs[0][:5], len(rev_seqs), rev_seqs[0][-5:], len(targets), targets[0]

# Compose different data transforms for this particular load
data_transform = transforms.Compose([
    ToTensor(transpose=False)
])

# Instantiate a Dataset
test_dataset = SeqDataset(seqs, names=names, targets=targets, rev_seqs=rev_seqs, transform=data_transform)

# Check the Dataset Class
for i in range(len(test_dataset)):
    sample = test_dataset[i]
    print(i, sample[0].size(), sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

# Instantiate a DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Check the DataLoader
for i_batch, sample_batched in enumerate(test_dataloader):
    print(i_batch, sample_batched[1].size(), sample_batched[2].size(), sample_batched[3].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

# Build SeqDataModule 
test_datamod = SeqDataModule(NUMPY_OHE)

(100,
 'seq001',
 100,
 array([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0]], dtype=int8),
 100,
 array([[0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0]], dtype=int8),
 100,
 0)

0 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
1 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
2 torch.Size([32, 66, 4]) torch.Size([32, 66, 4]) torch.Size([32])
3 torch.Size([4, 66, 4]) torch.Size([4, 66, 4]) torch.Size([4])


# Loading variable length sequences

In [126]:
seq_file="/cellar/users/aklie/projects/EUGENE/data/All_Genomic_Sequences/All_Genomic_Sequences.tsv"

In [127]:
import pandas as pd
pd.read_csv(seq_file, sep="\t").head()

,NAME,SEQ,FXN_LABEL,TILE,SEQ_LEN,DATASET,FXN_DESCRIPTION
0,scaffold_1:462149:462232,AAAGTAGGCTATATGCTACAGCCCAGAGCTCATGGATTTTAATGGG...,1,full,195,2010_Khoueiry_CellPress,NaN
1,scaffold_102:102675:102754,AAAGTAGGCTATCGCGACCACTGATCGTCGCGTAATTATTTTGAGG...,1,full,183,2010_Khoueiry_CellPress,NaN
2,scaffold_3:475040:475120,AAAGTAGGCTTTGTATAATTCCCAATTAGAACACACAGGTAGGGTA...,1,full,180,2010_Khoueiry_CellPress,NaN
3,scaffold_3:781073:781145,AAAGTAGGCTTTAACATGGAATCTATTCCCCGTGACGATGAGATAA...,1,full,154,2010_Khoueiry_CellPress,NaN
4,scaffold_357:46416:46496,AAAGTAGGCTTCTTGCTTTGTTGTTTTTGTTTTGAAACTGGTGAGC...,0,full,186,2010_Khoueiry_CellPress,NaN


In [145]:
from eugene.dataloading.load_data import load

In [146]:
# Load the sequences as numpy arrays
names, seqs, rev_seqs, targets = load(seq_file, seq_col="SEQ", name_col="NAME", target_col="FXN_LABEL", rev_comp=True)
len(names), names[0], len(seqs), seqs[0][:5], len(rev_seqs), rev_seqs[0][-5:], len(targets), targets[0]

(42, 'scaffold_1:462149:462232', 42, 'AAAGT', 42, 'ACTTT', 42, 1.0)

In [147]:
test_dataset = SeqDataset(seqs, names=names, targets=targets, rev_seqs=rev_seqs, transform=None)
test_dataset[0]

array([array([115.,  99.,  97., 102., 102., 111., 108., 100.,  95.,  49.,  58.,
               52.,  54.,  50.,  49.,  52.,  57.,  58.,  52.,  54.,  50.,  50.,
               51.,  50.,  36.,  36.])                                         ,
       'AAAGTAGGCTATATGCTACAGCCCAGAGCTCATGGATTTTAATGGGATCGGCTATCTAGGCCGACCCTCGCTCTCCCAAGGAAATGTCCACCTTCCAGCCGGGAAAAGATAACCGCTCGCCAGAGCGACGCTTTCCGGCTGACAAATTGTGTCGGACCTTGATAGCATTCCTGTTCCCTATCGGACCCAACTTT ',
       ' AAAGTTGGGTCCGATAGGGAACAGGAATGCTATCAAGGTCCGACACAATTTGTCAGCCGGAAAGCGTCGCTCTGGCGAGCGGTTATCTTTTCCCGGCTGGAAGGTGGACATTTCCTTGGGAGAGCGAGGGTCGGCCTAGATAGCCGATCCCATTAAAATCCATGAGCTCTGGGCTGTAGCATATAGCCTACTTT',
       1.0], dtype=object)

In [148]:
test_dataset[0][0]

array([115.,  99.,  97., 102., 102., 111., 108., 100.,  95.,  49.,  58.,
        52.,  54.,  50.,  49.,  52.,  57.,  58.,  52.,  54.,  50.,  50.,
        51.,  50.,  36.,  36.])

In [149]:
test_str = "".join([chr(int(letter)) for letter in test_dataset[0][0]]).replace("$", "")

In [153]:
test_strs = [ascii_decode(thing) for thing in test_dataset.ascii_names]

array(['scaffold_1:462149:462232', 'scaffold_102:102675:102754',
       'scaffold_3:475040:475120', 'scaffold_3:781073:781145',
       'scaffold_357:46416:46496', 'scaffold_577:39771:39829',
       'scaffold_6:512297:512370', 'scaffold_98:245430:245507',
       'scaffold_248:118347:118482', 'Scaffold_31:173924:174038',
       'scaffold_6:562995:563124', 'scaffold_11:490974:491037',
       'scaffold_366:40184:40258', 'scaffold_37:273449:273512',
       'scaffold_371:68262:68340', 'scaffold_40:155281:155360',
       'scaffold_48:226447:226527', 'scaffold_490:18446:18523',
       'scaffold_50:51150:51235', 'scaffold_522:36260:36327', 'Otxa-WTg1',
       'Otxa-WTg2', 'Otxa-WTg3', 'Otxa-WTg5', 'Negative_control', 'nc2',
       'nc3', 'nc4', 'nc5', 'EM.Ci1', 'EM.Dr1', 'EM.Ggd1', 'EM.Mm6',
       'EM.Hs9', 'EM.Mm2', 'EM.Hs1', 'EM.Hs2', 'EM.Hs3', 'EM.Hs8',
       'EM.Mm1', 'EM.Hs4', 'EM.Dr2'], dtype='<U26')

In [158]:
np.array(test_strs) == test_dataset.names

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
for asci in 

In [151]:
test_str

'scaffold_1:462149:462232'

In [91]:
test_dataset.ascii_names.shape

(42, 26)

In [92]:
np.any(test_dataset.name_lengths == test_dataset.name_lengths[0])

True

In [93]:
test_dataset.name_lengths

array([24, 26, 24, 24, 24, 24, 24, 25, 26, 25, 24, 25, 24, 25, 24, 25, 25,
       24, 23, 24,  9,  9,  9,  9, 16,  3,  3,  3,  3,  6,  6,  7,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6])

In [9]:
import numpy as np

In [51]:
test_dataset.longest_name

AttributeError: 'SeqDataset' object has no attribute 'longest_name'

In [107]:
from eugene.utils.seq_utils import ascii_encode, ascii_decode

In [122]:
asci_2 = np.zeros((len(test_dataset.names), test_dataset.longest_name))
for i, name in enumerate(test_dataset.names):
    pad_len = test_dataset.longest_name-len(name)
    asci_2[i] = ascii_encode(name, pad=pad_len)
    #np_name = np.array([ord(letter) for letter in name], dtype=int)
    #asci[i] = np.pad(np_name, pad_width=(0, pad_len), mode="constant", constant_values=36)

In [121]:
ascii_encode(name)

array([ 69,  77,  46,  68, 114,  50])

In [125]:
np.all(asci == asci_2)

True

In [114]:
asci.shape

(42, 26)

In [123]:
asci_2.shape

(42, 26)

In [49]:
test_dataset.names

array(['scaffold_1:462149:462232', 'scaffold_102:102675:102754',
       'scaffold_3:475040:475120', 'scaffold_3:781073:781145',
       'scaffold_357:46416:46496', 'scaffold_577:39771:39829',
       'scaffold_6:512297:512370', 'scaffold_98:245430:245507',
       'scaffold_248:118347:118482', 'Scaffold_31:173924:174038',
       'scaffold_6:562995:563124', 'scaffold_11:490974:491037',
       'scaffold_366:40184:40258', 'scaffold_37:273449:273512',
       'scaffold_371:68262:68340', 'scaffold_40:155281:155360',
       'scaffold_48:226447:226527', 'scaffold_490:18446:18523',
       'scaffold_50:51150:51235', 'scaffold_522:36260:36327', 'Otxa-WTg1',
       'Otxa-WTg2', 'Otxa-WTg3', 'Otxa-WTg5', 'Negative_control', 'nc2',
       'nc3', 'nc4', 'nc5', 'EM.Ci1', 'EM.Dr1', 'EM.Ggd1', 'EM.Mm6',
       'EM.Hs9', 'EM.Mm2', 'EM.Hs1', 'EM.Hs2', 'EM.Hs3', 'EM.Hs8',
       'EM.Mm1', 'EM.Hs4', 'EM.Dr2'], dtype='<U26')

In [47]:
asci.shape

(42, 26)

In [36]:
np_name

array([115,  99,  97, 102, 102, 111, 108, 100,  95,  49,  58,  52,  54,
        50,  49,  52,  57,  58,  52,  54,  50,  50,  51,  50])

In [41]:
test_dataset.longest_name

26

In [44]:
.shape

(26,)

In [20]:
np.pad(test_dataset.ascii_names, [(0, 0), (0, 0)], mode="constant", constant_values=-1)

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (1,2)

# MPRADataModule
PyTorch Lightning DataModule class for MPRA data that allows for asbtracting most of the dataloading process. These DataModules can be passed straight to trainers for model training.

In [77]:
from MPRADataModule import MPRADataModule

In [78]:
data_transform = transforms.Compose([
    Augment(randomize_linker_p=0.1, enhancer="WT-otx-a"), 
    OneHotEncode(), 
    ToTensor(transpose=True)
])

In [79]:
test_datamodule = MPRADataModule(
    seq_file="test_seqs.tsv",
    transform=data_transform,
    num_workers=0,
    batch_size=16,
    load_kwargs=dict(seq_col="SEQ"))

In [80]:
test_datamodule.setup()
test_dataset = test_datamodule.train_dataloader().dataset

In [81]:
# Check the Dataset Class
for i in range(len(test_datamodule.train_dataloader().dataset)):
    sample = test_datamodule.train_dataloader().dataset[i]
    print(i, sample[0].size(), sample[1].size(), sample[2].size(), sample[3])
    if i == 3:
        break

0 torch.Size([1]) torch.Size([4, 1000]) torch.Size([1]) tensor([-1.])
1 torch.Size([1]) torch.Size([4, 1000]) torch.Size([1]) tensor([-1.])
2 torch.Size([1]) torch.Size([4, 1000]) torch.Size([1]) tensor([-1.])
3 torch.Size([1]) torch.Size([4, 1000]) torch.Size([1]) tensor([-1.])


In [82]:
# Check the DataLoader
for i_batch, sample_batched in enumerate(test_datamodule.train_dataloader()):
    print(i_batch, sample_batched[1].size(), sample_batched[2].size(), sample_batched[3].size())
    # observe 4th batch and stop.
    if i_batch == 3:
        break

0 torch.Size([16, 4, 1000]) torch.Size([16, 1]) torch.Size([16, 1])
1 torch.Size([16, 4, 1000]) torch.Size([16, 1]) torch.Size([16, 1])
2 torch.Size([16, 4, 1000]) torch.Size([16, 1]) torch.Size([16, 1])
3 torch.Size([16, 4, 1000]) torch.Size([16, 1]) torch.Size([16, 1])


In [21]:
test_datamodule.

(tensor([-1.]),
 tensor([[0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]]),
 tensor([-1.]),
 tensor([-1.]))